In [11]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Azure AI Search tool from the Azure Agents service using a synchronous client.
    To learn how to set up an Azure AI Search resource,
    visit https://learn.microsoft.com/azure/search/search-get-started-portal

USAGE:
    Before running the sample:
    Set these environment variables with your own values:
    1) AZURE_AISTUDIO_PROJECT_CONN_STRING - The project connection string, as found in the overview page of your
       Azure AI Foundry project.
    2) AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in 
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_AI_SEARCH_CONNECTION_ID - The connection ID of the Azure AI Search resource, as found in the "Connections" tab
"""
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, AzureAISearchTool, ConnectionType, FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential
from typing import Any, Callable, Set, Dict, List, Optional
from pathlib import Path
from dotenv import load_dotenv
import sqlite3
import pandas as pd

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery"
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_AISTUDIO_PROJECT_CONN_STRING = os.getenv("AZURE_AISTUDIO_PROJECT_CONN_STRING")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=AZURE_AISTUDIO_PROJECT_CONN_STRING,
)

In [12]:
def get_conn(db_file: str) -> sqlite3.Connection:
    return sqlite3.connect(db_file)

def execute_query(conn: sqlite3.Connection, query: str) -> List[Any]:
    cursor = conn.cursor()
    return cursor.execute(query).fetchall()

def execute_query_pd(conn: sqlite3.Connection, query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)

# Get a description of a table into a pandas dataframe
def get_table_schema(conn: sqlite3.Connection, table_name: str) -> pd.DataFrame:
    query = f"PRAGMA table_info({table_name});"
    return pd.read_sql_query(query, conn)

#Construct a description of the DB schema for the LLM by retrieving the
# CREATE commands used to create the tables
def get_db_creation_sql(conn: sqlite3.Connection) -> str:
    cursor = conn.cursor()
    query = "SELECT sql FROM sqlite_master WHERE type='table'"
    results = cursor.execute(query).fetchall()
    results = [col[0] for col in results]
    schema_description = '\n'.join(results)
    return schema_description


def get_tables_from_db(conn: sqlite3.Connection) -> List[str]:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [x[0] for x in cursor.fetchall()]

In [13]:
database = 'bookstore.db'
conn = sqlite3.connect(database)
TABLE_NAME = 'Books'
schema = get_db_creation_sql(conn)
print(schema)

CREATE TABLE Authors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            biography TEXT
        )
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE Publishers (
            d INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            address TEXT
        )
CREATE TABLE Books (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            price REAL,
            author_id INTEGER NOT NULL, FOREIGN KEY (author_id) REFERENCES Authors(id)
        )


In [14]:
def read_from_db(sql_query:str) -> str:
    """
    run the query on the database and return the result

    :param sql_query (str): The sql to run on the database.
    :return: The query result
    """
    return execute_query_pd(conn, sql_query)

user_functions: Set[Callable[..., Any]] = {
    read_from_db
}

In [15]:
from typing import List, Optional, Tuple
import pandas as pd
import json

def get_schema_prompt(schema: str) -> str:
    prompt = 'Below is the information for an SQLite table.'
    prompt += f'Schema:\n{schema}\n\n'
    prompt += '\n------------------------------------------------------\n'
    return prompt

def get_prompt(schema: str) -> str:
    prompt = get_schema_prompt(schema)
    prompt += 'Below is a question input from a user. '
    prompt += 'Generate an SQL query that pulls the necessary data to answer the question.\n\n'
    return prompt

def format_result(question: str, query: str, explanation: str, result: str) -> str:
    return json.dumps(
        {
            'question': question,
            'query': query,
            'explanation': explanation,
            'result': result
        }, indent=4
    )

In [16]:
with project_client:
    # Initialize agent toolset with user functions and code interpreter
    functions = FunctionTool(user_functions)
    code_interpreter = CodeInterpreterTool()

    toolset = ToolSet()
    toolset.add(functions)
    toolset.add(code_interpreter)

    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME"],
        name="sql-assistant",
        instructions=get_prompt(schema),
        toolset=toolset,
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="How many books are in the bookstore?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_O1WUnpuGB6pinwEX7McueN7R
Created thread, ID: thread_U9yxMYG85YhTezOeXsjh6xKe
Created message, ID: msg_XwaU4FMSwX3zL8wBrAmxBA3a


TypeError: Object of type DataFrame is not JSON serializable

In [ ]:
# print(messages["data"][0]["content"][0]["text"]["value"])

# Fetch and log all messages in chronological order
messages_data = messages["data"]

# Sort messages by creation time (ascending)
sorted_messages = sorted(messages_data, key=lambda x: x["created_at"])

print("\n--- Thread Messages (sorted) ---")
for msg in sorted_messages:
    role = msg["role"].upper()
    # Each 'content' is a list; get the first text block if present
    content_blocks = msg.get("content", [])
    text_value = ""
    if content_blocks and content_blocks[0]["type"] == "text":
        text_value = content_blocks[0]["text"]["value"]
    print(f"{role}: {text_value}")